In [1]:
import sys
sys.path.append('../')
from plotting import CandlePlotting
import pandas as pd
import plotly.graph_objects as go


In [2]:
from tecniche.pattern import apply_pattern

df = pd.read_pickle('../data/AUD_CAD_M5.pkl')

In [3]:
df_an = df[['time', 'mid_c', 'mid_h', 'mid_l', 'mid_o']].copy()

In [4]:
df_an.tail()

,time,mid_c,mid_h,mid_l,mid_o
73673,2016-12-30 21:35:00+00:00,0.96806,0.96812,0.96795,0.96805
73674,2016-12-30 21:40:00+00:00,0.96862,0.96864,0.96804,0.96806
73675,2016-12-30 21:45:00+00:00,0.96908,0.96908,0.96860,0.96860
73676,2016-12-30 21:50:00+00:00,0.96913,0.96918,0.96884,0.96908
73677,2016-12-30 21:55:00+00:00,0.96791,0.96906,0.96712,0.96906


In [5]:
# direction = df_an.mid_c - df_an.mid_o

# body_size = abs(direction)

# direction = [1 if x >= 0 else -1 for x in direction]

# full_range = df_an.mid_h - df_an.mid_l
# body_perc = (body_size / full_range) * 100
# body_lower = df_an[['mid_c', 'mid_o']].min(axis=1)
# body_upper = df_an[['mid_c', 'mid_o']].max(axis=1)
# body_bottom_perc =(( body_lower - df_an.mid_l) / full_range) * 100
# body_top_perc =(( body_upper - df_an.mid_l) / full_range) * 100


In [6]:
# df_an['body_lower'] = body_lower

# df_an['body_upper'] = body_upper

# df_an['body_bottom_perc'] = body_bottom_perc

# df_an['body_top_perc'] = body_top_perc

# df_an['body_perc'] = body_perc

In [7]:
df_an = apply_pattern(df_an)

In [8]:
HANGING_MAN_BODY = 15.0
HANGING_MAN_HEIGHT = 75.0
SHOOTING_STAR_HEIGHT = 25.0
SPINNING_TOP_MIN = 40.0
SPINNING_TOP_MAX = 60.0
MARUBOZU = 98.0

ENGULFING_FACTOR = 1.1



apply_marubozu = lambda x: x.body_perc > MARUBOZU

def apply_hanging_man(row):
    if row.body_bottom_perc > HANGING_MAN_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY: 
            return True
    return False

def apply_shooting_star(row):
    if row.body_top_perc < SHOOTING_STAR_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY: 
            return True
    return False

def apply_spinning_top(row):
    if row.body_top_perc < SPINNING_TOP_MAX:
        if row.body_bottom_perc > SPINNING_TOP_MIN: 
            if row.body_perc < HANGING_MAN_BODY:
                return True
    return False

def apply_engulfing(row):
    if row.direction != row.direction_prev:
        if row.body_size > row.body_size_prev * ENGULFING_FACTOR:
            return True
    return False

TWEEZER_BODY = 15.0
TWEEZER_HL = 0.01
TWEEZER_TOP_BODY = 40.0
TWEEZER_BOTTOM_BODY = 60.0



def apply_tweezer_top(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == -1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_top_perc < TWEEZER_TOP_BODY:
                    return True
    return False    

def apply_tweezer_bottom(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == 1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_bottom_perc > TWEEZER_BOTTOM_BODY:
                    return True
    return False


MORNING_STAR_PREV2_BODY = 90.0
MORNING_STAR_PREV_BODY = 10.0

def apply_morning_star(row, direction=1):
    if row.body_perc_prev_2 > MORNING_STAR_PREV2_BODY:
            if row.body_perc_prev < MORNING_STAR_PREV_BODY:
                if row.direction == direction and row.direction_prev_2 != direction:
                    if direction == 1:
                        if row.mid_c > row.mid_point_prev_2:
                            return True
                    else:
                        if row.mid_c < row.mid_point_prev_2:
                            return True
    return False


In [9]:
df_an['body_size_prev'] = df_an.body_size.shift(1)
df_an['direction_prev'] = df_an.direction.shift(1)
df_an['direction_prev_2'] = df_an.direction.shift(2)
df_an['body_perc_prev'] = df_an.body_perc.shift(1)
df_an['body_perc_prev_2'] = df_an.body_perc.shift(2)



df_an['HANGING_MAN'] = df_an.apply(apply_hanging_man, axis=1)
df_an['SHOOTING_STAR'] = df_an.apply(apply_shooting_star, axis=1)
df_an['SPINNING_TOP'] = df_an.apply(apply_spinning_top, axis= 1)
df_an['MARUBOZU']= df_an.apply(apply_marubozu, axis = 1)
df_an['ENGULFING'] = df_an.apply(apply_engulfing, axis = 1)
df_an['TWEEZER_TOP'] = df_an.apply(apply_tweezer_top, axis = 1)
df_an['TWEEZER_BOTTOM'] = df_an.apply(apply_tweezer_bottom, axis = 1)

df_an['MORNING_STAR'] = df_an.apply(apply_morning_star , axis = 1)
df_an['EVENING_STAR'] = df_an.apply(apply_morning_star , axis = 1, direction = -1)


In [14]:
df_an[df_an['EVENING_STAR'] == True]

,time,mid_c,mid_h,mid_l,mid_o,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
6672,2016-02-09 04:00:00+00:00,0.97997,0.98167,0.97997,0.98122,0.97997,0.98122,0.000000,73.529412,73.529412,...,100.000000,False,False,False,False,True,False,False,False,True
7489,2016-02-12 00:05:00+00:00,0.99004,0.99159,0.98998,0.99154,0.99004,0.99154,3.726708,96.894410,93.167702,...,93.333333,False,False,False,False,True,False,False,False,True
8632,2016-02-17 23:55:00+00:00,0.98126,0.98170,0.98111,0.98156,0.98126,0.98156,25.423729,76.271186,50.847458,...,91.489362,False,False,False,False,True,False,False,False,True
9529,2016-02-23 02:40:00+00:00,0.99156,0.99218,0.99140,0.99218,0.99156,0.99218,20.512821,100.000000,79.487179,...,95.774648,False,False,False,False,False,False,False,False,True
12566,2016-03-08 15:45:00+00:00,0.99609,0.99674,0.99582,0.99673,0.99609,0.99673,29.347826,98.913043,69.565217,...,91.358025,False,False,False,False,False,False,False,False,True
14824,2016-03-18 10:55:00+00:00,0.99062,0.99175,0.99062,0.99163,0.99062,0.99163,0.000000,89.380531,89.380531,...,100.000000,False,False,False,False,True,False,False,False,True
14879,2016-03-18 15:30:00+00:00,0.98779,0.98861,0.98770,0.98834,0.98779,0.98834,9.890110,70.329670,60.439560,...,98.113208,False,False,False,False,True,False,False,False,True
15529,2016-03-22 21:40:00+00:00,0.99364,0.99403,0.99358,0.99400,0.99364,0.99400,13.333333,93.333333,80.000000,...,96.551724,False,False,False,False,False,False,False,False,True
16112,2016-03-24 22:15:00+00:00,0.99612,0.99756,0.99556,0.99668,0.99612,0.99668,28.000000,56.000000,28.000000,...,100.000000,False,False,False,False,False,False,False,False,True
19588,2016-04-11 23:55:00+00:00,0.98022,0.98058,0.98022,0.98049,0.98022,0.98049,0.000000,75.000000,75.000000,...,97.435897,False,False,False,False,False,False,False,False,True


In [16]:
MARKER = '#0066FF'

dfp = df_an.iloc[6600:6800]
cp = CandlePlotting(dfp, candele=True)
df_temp = cp.df_plot[cp.df_plot.EVENING_STAR == True]

cp.fig.add_trace(go.Candlestick(
                x=df_temp.sTime,
                open=df_temp.mid_o,
                high=df_temp.mid_h,
                low=df_temp.mid_l,
                close=df_temp.mid_c,
                line=dict(width=1),
                opacity=1,
                increasing_fillcolor=MARKER,
                decreasing_fillcolor=MARKER,
                increasing_line_color=MARKER,
                decreasing_line_color=MARKER
            ))
cp.show_plot()